In [24]:
import numpy as np


def levenshtein(x, y):
    
    """
    levenshtein distance for iterable sequences
    """
    
    # check type
    if (np.all(map(type, x)) is str) and (np.all(map(type, y)) is str):
        _x = np.array(x, dtype=np.str)
        _y = np.array(y, dtype=np.str)
    elif (np.all(map(type, x)) is int) and (np.all(map(type, y)) is int):
        _x = np.array(x, dtype=np.int)
        _y = np.array(y, dtype=np.int)
    elif type(x) is str and type(y) is str:
        _x = np.array(list(x), dtype=np.str)
        _y = np.array(list(y), dtype=np.str)
    else:
        raise TypeError

    d, D = _levenshtein(_x, _y)
    return d, D


def _levenshtein(x, y):
    
    """ Levenshtein distance
          using Dynamic-Programming strategy
    Parameters
    ----------
    x, y : np.array of string
    Returns
    -------
    int : distance
    np.array : distance matrix
    """
    
    # Initiallize DP-matrix
    D = np.zeros((len(x) + 1, len(y) + 1), dtype=int)
    D[0, 1:] = range(1, len(y) + 1)
    D[1:, 0] = range(1, len(x) + 1)

    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            delta = 1 if x[i - 1] != y[j - 1] else 0
            D[i, j] = min(D[i - 1, j - 1] + delta, D[i - 1, j] + 1, D[i, j - 1] + 1)
    return D[-1, -1], D


def backtrace(x, y, D):
    
    """ Get alignment for given sequences and Distance-Matrix
    Parameters
    ----------
    x, y : np.array or array-like
    D : np.array 
        Distance matrix computed by `levenshtein`
    Returns
    -------
    tuple : np.array or array-like * 3
        t[0], t[1] are annotated sequence corresponding to x, y
        t[2] is possible edit
    """
    
    edit = []
    _x = []
    _y = []
    i = len(x)
    j = len(y)
    mincost = np.inf
    prevcost = D[i, j]

    while not (i == 0 and j == 0):

        direction = np.argmin([D[i - 1, j - 1], D[i, j - 1], D[i - 1, j]])
        mincost = np.min([D[i - 1, j - 1], D[i, j - 1], D[i - 1, j]])

        if direction == 0:
            if mincost == prevcost:
                edit.append(" ")  # match (x[i-1] == y[j-1])
            else:
                edit.append("*")  # substitution
            _x.append(x[i - 1])
            _y.append(y[j - 1])
            i -= 1
            j -= 1
        elif direction == 1:
            edit.append("+")  # insertion (to x)
            _x.append(" ")
            _y.append(y[j - 1])
            j -= 1
        elif direction == 2:
            edit.append("-")  # deletion (from x)
            _x.append(x[i - 1])
            _y.append(" ")
            i -= 1

        prevcost = mincost

    edit.reverse()
    _x.reverse()
    _y.reverse()
    return _x, _y, edit


def _test(s1, s2, expected_ed: int = None):
    d, D = levenshtein(s1, s2)
    print("S1 = {}".format(s1))
    print("S2 = {}".format(s2))
    print("Distance = {}\n".format(d))

    _s1, _s2, edit = backtrace(s1, s2, D)
    print("Alignment")
    for a, b, op in zip(_s1, _s2, edit):
        print("    {} {} {}".format(a, op, b))
    print("\n")

    if expected_ed:
        assert d == expected_ed and edit is not None
    else:
        assert edit is not None


def test_basic():
    x = "занятию"
    y = "занятие"
    _test(x, y)


if __name__ == '__main__':
    tests = [test_basic, ]

    for testcase in tests:
        try:
            print("{}".format(testcase.__name__))
            print("-" * len(testcase.__name__))
            testcase()
        except AssertionError:
            print("FAILED: testcase= {}, cause=AssertionError".format(testcase.__name__))
        except Exception as e:
            print("FAILED: testcase= {}, cause={}".format(testcase.__name__, str(e)))
        finally:
            print("")

test_basic
----------
S1 = занятию
S2 = занятие
Distance = 1

Alignment
    з   з
    а   а
    н   н
    я   я
    т   т
    и   и
    ю * е





In [44]:
def dtw(s1: list, s2: list) -> int:
    
    mat = [
        [0 for _ in range(len(s2))]
        for _ in range(len(s1))
    ]
    
    mat[0][0] = abs(s1[0] - s2[0])
    
    for i in range(1, len(s1)):
        mat[i][0] = mat[i - 1][0] + abs(s1[i] - s2[0])
        
    for j in range(1, len(s2)):
        mat[0][j] = mat[0][j - 1] + abs(s1[0] - s2[j])
        
    
    for i in range(1, len(s1)):
        for j in range(1, len(s2)):
            
            mat[i][j] = min(
                (mat[i - 1][j], mat[i][j - 1], mat[i - 1][j - 1])
            ) + abs(s1[i] - s2[j])
            
    return mat

In [45]:
from pandas import DataFrame

DataFrame(dtw(
    [3, -13, 14, -7, 9, -2],
    [-2, 10, -10, 15, -13, 20, -5, 14, 2],
))

,0,1,2,3,4,5,6,7,8
0,5,12,25,37,53,70,78,89,90
1,16,35,28,56,53,86,86,113,105
2,32,36,52,53,80,86,105,105,117
3,37,53,55,75,81,108,107,126,126
4,48,49,68,74,96,107,121,126,133
5,48,60,68,85,96,118,121,137,137


In [40]:
DataFrame(dtw(
    [6, 6, 4, 4, 0, 2, 0],
    [1, 3, 2, 8, 8, 0, 4, 0],
))

,0,1,2,3,4,5,6,7
0,5,8,12,14,16,22,24,30
1,10,8,12,14,16,22,24,30
2,13,9,10,14,18,20,20,24
3,16,10,11,14,18,22,20,24
4,17,13,12,19,22,18,22,20
5,18,14,12,18,24,20,20,22
6,19,17,14,20,26,20,24,20
